In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# ============ Key-XOR Layer (Explicit Key Operation) ============
class KeyXORLayer(nn.Module):
    """
    Applies key-conditioned XOR-like operation
    This ensures encryption DEPENDS on keys
    """
    def __init__(self, dim):
        super(KeyXORLayer, self).__init__()
        # Learnable projection for keys
        self.key_proj = nn.Linear(1, dim)
        self.mixing = nn.Linear(dim, dim)

    def forward(self, x, key):
        """
        x: (batch, seq_len, dim)
        key: (batch, seq_len)
        """
        # Project key to same dimension as x
        key_expanded = key.unsqueeze(-1)  # (batch, seq_len, 1)
        key_features = self.key_proj(key_expanded)  # (batch, seq_len, dim)

        # XOR-like operation (element-wise multiplication + residual)
        xor_out = x * torch.tanh(key_features)
        mixed = self.mixing(xor_out)

        return x + mixed  # Residual connection


# ============ Residual Convolution Block ============
class ResidualConvBlock(nn.Module):
    """Conv block with residual connection"""
    def __init__(self, channels, kernel_size=3):
        super(ResidualConvBlock, self).__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size, padding=kernel_size//2)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size, padding=kernel_size//2)
        self.norm1 = nn.LayerNorm(channels)
        self.norm2 = nn.LayerNorm(channels)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        """x: (batch, channels, seq_len)"""
        residual = x

        # First conv
        out = self.conv1(x)
        out = out.permute(0, 2, 1)  # (batch, seq_len, channels)
        out = self.norm1(out)
        out = out.permute(0, 2, 1)  # back to (batch, channels, seq_len)
        out = F.relu(out)
        out = self.dropout(out)

        # Second conv
        out = self.conv2(out)
        out = out.permute(0, 2, 1)
        out = self.norm2(out)
        out = out.permute(0, 2, 1)

        # Residual
        return F.relu(out + residual)


# ============ Simplified Alice Encoder ============
class SimplifiedAliceEncoder(nn.Module):
    """
    Alice: Encodes and encrypts messages using keys
    Key innovation: Explicit key-XOR operations at each layer
    """
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=3, max_len=128):
        super(SimplifiedAliceEncoder, self).__init__()

        self.embed_dim = embed_dim
        self.max_len = max_len

        # Character embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)

        # Initial projection
        self.input_proj = nn.Linear(embed_dim, embed_dim)

        # Encoding layers with key-XOR
        self.conv_layers = nn.ModuleList([
            ResidualConvBlock(embed_dim) for _ in range(num_layers)
        ])

        self.key_xor_layers = nn.ModuleList([
            KeyXORLayer(embed_dim) for _ in range(num_layers)
        ])

        # Output projection
        self.output_proj = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.LayerNorm(embed_dim),
            nn.Tanh()
        )

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for module in self.modules():
            if isinstance(module, (nn.Linear, nn.Embedding)):
                module.weight.data.normal_(mean=0.0, std=0.02)
                if isinstance(module, nn.Linear) and module.bias is not None:
                    module.bias.data.zero_()

    def forward(self, tokens, keys):
        """
        tokens: (batch, seq_len) - integer indices
        keys: list of 4 tensors, each (batch, seq_len)
        Returns: (batch, seq_len, embed_dim) - encrypted embeddings
        """
        batch_size, seq_len = tokens.size()

        # Embed tokens
        x = self.embedding(tokens)  # (batch, seq_len, embed_dim)
        x = self.input_proj(x)

        # Process through layers with key conditioning
        for i, (conv_layer, key_xor) in enumerate(zip(self.conv_layers, self.key_xor_layers)):
            # Convolutional encoding
            x_conv = x.permute(0, 2, 1)  # (batch, embed_dim, seq_len)
            x_conv = conv_layer(x_conv)
            x = x_conv.permute(0, 2, 1)  # (batch, seq_len, embed_dim)

            # Apply key-XOR (this makes encryption depend on keys!)
            x = key_xor(x, keys[i])

        # Final projection
        encrypted = self.output_proj(x)

        return encrypted


# ============ Alice Wrapper ============
class AliceEncryptor:
    """Wrapper for Alice encoder"""
    def __init__(self, vocab_size=98, embed_dim=128, num_layers=3, max_len=128,
                 device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.max_len = max_len
        self.alice = SimplifiedAliceEncoder(vocab_size, embed_dim, num_layers, max_len).to(device)

        from v2_part1_simplified import StringProcessor
        self.processor = StringProcessor(max_length=max_len)

    def encrypt(self, messages, keys):
        """
        Encrypt messages (strings or tokens)
        messages: list of strings OR tensor of tokens
        keys: dict from KeyManager with 'key_tensors'
        """
        # Convert strings to tokens if needed
        if isinstance(messages, list) and isinstance(messages[0], str):
            tokens = self.processor.batch_encode(messages).to(self.device)
        else:
            tokens = messages.to(self.device)

        # Truncate keys to match sequence length
        seq_len = tokens.size(1)
        truncated_keys = [k[:, :seq_len] for k in keys['key_tensors']]

        # Encrypt
        self.alice.eval()
        with torch.no_grad():
            encrypted = self.alice(tokens, truncated_keys)

        return encrypted


# ============ Usage Example ============
if __name__ == "__main__":
    print("="*70)
    print("SIMPLIFIED ALICE ENCODER V2")
    print("="*70)

    # Initialize
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    from v2_part1_simplified import StringProcessor, KeyManager

    processor = StringProcessor(max_length=64)
    key_manager = KeyManager(device=device)
    alice_enc = AliceEncryptor(
        vocab_size=processor.vocab_size,
        embed_dim=128,
        num_layers=3,
        max_len=64,
        device=device
    )

    # Test messages
    messages = [
        "Hello World!",
        "Testing encryption system.",
        "Short test"
    ]

    print(f"\nTest messages:")
    for i, msg in enumerate(messages):
        print(f"  {i+1}. '{msg}'")

    # Generate keys
    keys = key_manager.generate_keys_for_batch(
        batch_size=len(messages),
        seq_length=64
    )

    # Encrypt
    print("\nEncrypting...")
    encrypted = alice_enc.encrypt(messages, keys)

    print(f"\nEncryption complete!")
    print(f"  Input: {len(messages)} messages")
    print(f"  Output shape: {encrypted.shape}")
    print(f"  Sample encrypted values: {encrypted[0, 0, :10]}")

    print("\n" + "="*70)

SIMPLIFIED ALICE ENCODER V2

Test messages:
  1. 'Hello World!'
  2. 'Testing encryption system.'
  3. 'Short test'

Encrypting...

Encryption complete!
  Input: 3 messages
  Output shape: torch.Size([3, 64, 128])
  Sample encrypted values: tensor([-0.4267, -0.5409, -0.4186, -0.3193,  0.5138,  0.2367, -0.3433, -0.9630,
         0.1794, -0.2162])

